Create a graph database in Neo4j for the BART system

In [1]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd

import psycopg2

In [2]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [3]:
session = driver.session(database="neo4j")

In [4]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [5]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [6]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")


In [7]:
def my_neo4j_nodes_relationships():
    "print all the nodes and relationships"
   
    print("-------------------------")
    print("  Nodes:")
    print("-------------------------")
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    display(df)
    
    print("-------------------------")
    print("  Relationships:")
    print("-------------------------")
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    display(df)
    
    density = (2 * number_relationships) / (number_nodes * (number_nodes - 1))
    
    print("-------------------------")
    print("  Density:", f'{density:.1f}')
    print("-------------------------")
    

In [8]:
def my_neo4j_nodes():
    "print all the nodes and relationships"
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    new_df = pd.DataFrame(df)
    
    return(new_df)

In [9]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [10]:
def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    

In [11]:
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)

In [12]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [13]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [14]:
cursor = connection.cursor()

In [15]:
my_neo4j_wipe_out_database()

In [16]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------


In [17]:
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node(station)
    

In [18]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 50
  Relationships: 0
-------------------------


In [19]:
connection.rollback()

query = """

select*
from travel_times

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    from_station = row[1]
    to_station = row[0]
    travel_time = int(row[2])
    
    my_neo4j_create_relationship_two_way(from_station, to_station, travel_time)
    

In [20]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 50
  Relationships: 102
-------------------------


In [21]:
connection.rollback()

query = """

select a.line, a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
order by line, from_station, to_station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    line = row[0]
    from_station = row[1]
    to_station = row[2]
    travel_time = int(row[3])
    
    my_neo4j_create_relationship_two_way(from_station, to_station, travel_time)
    

In [22]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 50
  Relationships: 318
-------------------------


Centrality Algorithms 

In [23]:
#Degree Centrality of BART stations
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK')"
session.run(query)

In [24]:
query = """

CALL gds.degree.stream('ds_graph')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score as degree
ORDER BY degree DESC, name

"""

degree_centrality_df = my_neo4j_run_query_pandas(query)

In [25]:
degree_centrality_df.head(50)

,name,degree
0,West Oakland,11.0
1,16th Street Mission,10.0
2,24th Street Mission,10.0
3,Balboa Park,10.0
4,Civic Center,10.0
5,Coliseum,10.0
6,Embarcadero,10.0
7,Glen Park,10.0
8,Montgomery Street,10.0
9,Powell Street,10.0


In [26]:
#Closeness Centrality of BART stations

query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK')"
session.run(query)

In [27]:
query = """

CALL gds.beta.closeness.stream('ds_graph')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score as closeness
ORDER BY score DESC

"""

closeness_centrality_df = my_neo4j_run_query_pandas(query)


In [28]:
closeness_centrality_df.head(50)

,name,closeness
0,12th Street,0.147147
1,Lake Merritt,0.145833
2,West Oakland,0.145401
3,19th Street,0.141210
4,Fruitvale,0.137640
5,Embarcadero,0.136490
6,MacArthur,0.134986
7,Coliseum,0.129630
8,Montgomery Street,0.127937
9,Rockridge,0.124682


In [29]:
#Harmonic Centrality of BART stations
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK')"
session.run(query)

In [30]:
query = """

CALL gds.alpha.closeness.harmonic.stream('ds_graph', {})
YIELD nodeId, centrality
RETURN gds.util.asNode(nodeId).name AS name, centrality as closeness
ORDER BY centrality DESC

"""

harmonic_centrality_df = my_neo4j_run_query_pandas(query)
harmonic_centrality_df.rename(columns={'closeness': 'harmonic'}, inplace=True)


In [31]:
harmonic_centrality_df.head(50)

,name,harmonic
0,12th Street,0.233302
1,Lake Merritt,0.231867
2,West Oakland,0.228991
3,MacArthur,0.226052
4,19th Street,0.219525
5,Coliseum,0.216537
6,Fruitvale,0.214852
7,Bay Fair,0.209259
8,Embarcadero,0.204956
9,San Leandro,0.204085


In [32]:
#Betweenness Centrality of BART stations

query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK')"
session.run(query)

In [33]:
query = """

CALL gds.betweenness.stream('ds_graph')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score as betweenness
ORDER BY betweenness DESC

"""

betweenness_centrality_df = my_neo4j_run_query_pandas(query)


In [34]:
betweenness_centrality_df.head(50)

,name,betweenness
0,MacArthur,1176.0
1,12th Street,1116.0
2,19th Street,1088.0
3,Lake Merritt,1020.0
4,Fruitvale,980.0
5,West Oakland,980.0
6,Coliseum,960.0
7,Embarcadero,936.0
8,Montgomery Street,888.0
9,Powell Street,836.0


In [35]:
#Louvain Modularity

query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = """

CALL gds.graph.project('ds_graph', 'Station', 'LINK')
"""

session.run(query)

In [36]:
query = """

CALL gds.louvain.stream('ds_graph')
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).name AS name, communityId as community, intermediateCommunityIds as intermediate_community
ORDER BY community, name ASC

"""

community_df = my_neo4j_run_query_pandas(query)
community_df

,name,community,intermediate_community
0,16th Street Mission,14,None
1,24th Street Mission,14,None
2,Balboa Park,14,None
3,Daly City,14,None
4,Glen Park,14,None
5,12th Street,15,None
6,19th Street,15,None
7,Ashby,15,None
8,Downtown Berkeley,15,None
9,El Cerrito Plaza,15,None


In [37]:
grouped_community_df = community_df.groupby('community')
for group_name, group_data in grouped_community_df:
    print(f"Community: {group_name}")
    print(group_data)
    print("----------------------------------")

Community: 14
                  name  community intermediate_community
0  16th Street Mission         14                   None
1  24th Street Mission         14                   None
2          Balboa Park         14                   None
3            Daly City         14                   None
4            Glen Park         14                   None
----------------------------------
Community: 15
                    name  community intermediate_community
5            12th Street         15                   None
6            19th Street         15                   None
7                  Ashby         15                   None
8      Downtown Berkeley         15                   None
9       El Cerrito Plaza         15                   None
10  El Cerrito del Norte         15                   None
11             MacArthur         15                   None
12        North Berkeley         15                   None
13              Richmond         15                   None
-----

In [38]:
# Merge degree_centrality_df and closeness_centrality_df
main_df = pd.merge(degree_centrality_df, closeness_centrality_df, on='name', how='outer')

# Merge harmonic_centrality_df and betweenness_centrality_df
main_df = pd.merge(main_df, harmonic_centrality_df, on='name', how='outer')

# Merge with betweenness_centrality_df
main_df = pd.merge(main_df, betweenness_centrality_df, on='name', how='outer')

# Merge with community_df
main_df = pd.merge(main_df, community_df, on='name', how='outer')

main_df


,name,degree,closeness,harmonic,betweenness,community,intermediate_community
0,West Oakland,11.0,0.145401,0.228991,980.0,29,None
1,16th Street Mission,10.0,0.104925,0.171228,720.0,14,None
2,24th Street Mission,10.0,0.098196,0.166316,656.0,14,None
3,Balboa Park,10.0,0.086116,0.157391,516.0,14,None
4,Civic Center,10.0,0.112128,0.176807,780.0,29,None
5,Coliseum,10.0,0.129630,0.216537,960.0,25,None
6,Embarcadero,10.0,0.136490,0.204956,936.0,29,None
7,Glen Park,10.0,0.091932,0.161775,588.0,14,None
8,Montgomery Street,10.0,0.127937,0.192243,888.0,29,None
9,Powell Street,10.0,0.119804,0.183524,836.0,29,None


In [39]:
# Remove all station names with 0 results on centrality algo and community detection
main_clean_df = main_df[main_df['degree'] != 0]
# Drop column 'intermediate_commuinty'
main_clean_df = main_clean_df.drop('intermediate_community', axis=1)

main_clean_df

,name,degree,closeness,harmonic,betweenness,community
0,West Oakland,11.0,0.145401,0.228991,980.0,29
1,16th Street Mission,10.0,0.104925,0.171228,720.0,14
2,24th Street Mission,10.0,0.098196,0.166316,656.0,14
3,Balboa Park,10.0,0.086116,0.157391,516.0,14
4,Civic Center,10.0,0.112128,0.176807,780.0,29
5,Coliseum,10.0,0.129630,0.216537,960.0,25
6,Embarcadero,10.0,0.136490,0.204956,936.0,29
7,Glen Park,10.0,0.091932,0.161775,588.0,14
8,Montgomery Street,10.0,0.127937,0.192243,888.0,29
9,Powell Street,10.0,0.119804,0.183524,836.0,29


Finding Total population within 1 mile of station

In [40]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [41]:
import psycopg2
from geographiclib.geodesic import Geodesic

def my_station_get_zips(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    total_population = 0
    
    for row in rows:
        zip = row[0]
        population = row[1]
        total_population += population
        
    
    return total_population


In [42]:
# Create column for population_within_1mile of each station
main_clean_df['population_within_1mile'] = main_clean_df['name'].apply(my_station_get_zips, args=(1,))

main_clean_df

,name,degree,closeness,harmonic,betweenness,community,population_within_1mile
0,West Oakland,11.0,0.145401,0.228991,980.0,29,26254
1,16th Street Mission,10.0,0.104925,0.171228,720.0,14,63489
2,24th Street Mission,10.0,0.098196,0.166316,656.0,14,108915
3,Balboa Park,10.0,0.086116,0.157391,516.0,14,106589
4,Civic Center,10.0,0.112128,0.176807,780.0,29,74898
5,Coliseum,10.0,0.129630,0.216537,960.0,25,0
6,Embarcadero,10.0,0.136490,0.204956,936.0,29,54398
7,Glen Park,10.0,0.091932,0.161775,588.0,14,115068
8,Montgomery Street,10.0,0.127937,0.192243,888.0,29,85465
9,Powell Street,10.0,0.119804,0.183524,836.0,29,140730


Load Exist Number

In [43]:
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [44]:
def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

In [45]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [46]:
cursor = connection.cursor()

In [47]:
connection.rollback()

query = """

drop table if exists station_entries_exits; 

"""

cursor.execute(query)

connection.commit()


In [48]:
connection.rollback()

query = """

create table station_entries_exits (
  station varchar(100),
  station_name varchar(100),
  entries numeric(12),
  exits numeric(12)
);

"""

cursor.execute(query)

connection.commit()



In [49]:
my_read_csv_file("station_entries_exits.csv", limit=10)

['station', 'station_name', 'Entries', 'Exits']
['RM', 'Richmond', '58940', '55853']
['EN', 'El Cerrito del Norte', '87642', '90217']
['EP', 'El Cerrito Plaza', '49852', '49632']
['NB', 'North Berkeley', '46950', '44942']
['BK', 'Downtown Berkeley', '118302', '123180']
['AS', 'Ashby', '51830', '49312']
['MA', 'MacArthur', '94830', '91758']
['19', '19th Street', '113251', '107926']
['12', '12th Street', '119623', '115837']

Printed  10 lines of  51 total lines.


In [50]:
connection.rollback()

query = """

copy station_entries_exits
from '/user/projects/project-3-nicole12hk/code/station_entries_exits.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

In [51]:
#Table shows number of exits for each station

rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from station_entries_exits
order by exits desc

"""

station_entries_exits_df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
station_entries_exits_df

,station,station_name,entries,exits
0,EM,Embarcadero,370182,420668
1,MT,Montgomery Street,295434,336738
2,PL,Powell Street,337468,304277
3,CC,Civic Center,251308,234454
4,16,16th Street Mission,150447,154355
5,24,24th Street Mission,139558,138430
6,BK,Downtown Berkeley,118302,123180
7,12,12th Street,119623,115837
8,19,19th Street,113251,107926
9,DC,Daly City,97685,100488


In [52]:
#Add entries and exits columns from station_entries_exits
merged_df = pd.merge(main_clean_df, station_entries_exits_df, left_on='name', right_on='station_name', how='inner')
merged_df.drop(columns=['station', 'station_name'], inplace=True)

merged_df

,name,degree,closeness,harmonic,betweenness,community,population_within_1mile,entries,exits
0,West Oakland,11.0,0.145401,0.228991,980.0,29,26254,93142,84396
1,16th Street Mission,10.0,0.104925,0.171228,720.0,14,63489,150447,154355
2,24th Street Mission,10.0,0.098196,0.166316,656.0,14,108915,139558,138430
3,Balboa Park,10.0,0.086116,0.157391,516.0,14,106589,96422,87415
4,Civic Center,10.0,0.112128,0.176807,780.0,29,74898,251308,234454
5,Coliseum,10.0,0.129630,0.216537,960.0,25,0,80998,84167
6,Embarcadero,10.0,0.136490,0.204956,936.0,29,54398,370182,420668
7,Glen Park,10.0,0.091932,0.161775,588.0,14,115068,81193,78022
8,Montgomery Street,10.0,0.127937,0.192243,888.0,29,85465,295434,336738
9,Powell Street,10.0,0.119804,0.183524,836.0,29,140730,337468,304277


In [53]:
#Add community column from community_df
final_df = pd.merge(merged_df, community_df, left_on='name', right_on='name', how='inner')
final_df.rename(columns = {'community_y':'community'}, inplace = True)
final_df.drop(columns=['community_x', 'intermediate_community'], inplace=True)
final_df

#final_sort1 = final_df.sort_values(by=['exits'],ascending=[False])
#final_sort1.head(30)

,name,degree,closeness,harmonic,betweenness,population_within_1mile,entries,exits,community
0,West Oakland,11.0,0.145401,0.228991,980.0,26254,93142,84396,29
1,16th Street Mission,10.0,0.104925,0.171228,720.0,63489,150447,154355,14
2,24th Street Mission,10.0,0.098196,0.166316,656.0,108915,139558,138430,14
3,Balboa Park,10.0,0.086116,0.157391,516.0,106589,96422,87415,14
4,Civic Center,10.0,0.112128,0.176807,780.0,74898,251308,234454,29
5,Coliseum,10.0,0.129630,0.216537,960.0,0,80998,84167,25
6,Embarcadero,10.0,0.136490,0.204956,936.0,54398,370182,420668,29
7,Glen Park,10.0,0.091932,0.161775,588.0,115068,81193,78022,14
8,Montgomery Street,10.0,0.127937,0.192243,888.0,85465,295434,336738,29
9,Powell Street,10.0,0.119804,0.183524,836.0,140730,337468,304277,29


Find Closest BART station for Existing Customers

In [54]:
#Customers table

rollback_before_flag = True
rollback_after_flag = True

query = """

select a.*, b.latitude, b.longitude
from customers a, zip_codes b
where a.zip = b.zip
and a.state = 'CA'

"""

customer_zip = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
customer_zip

,customer_id,first_name,last_name,street,city,state,zip,closest_store_id,distance,latitude,longitude
0,1,Robb,Weaving,5 Ramsey Place,Oakland,CA,94609,1,1,37.8343,-122.2643
1,2,Robby,Belliard,6 Londonderry Plaza,Oakland,CA,94609,1,1,37.8343,-122.2643
2,3,Sadella,Caudrelier,548 Mcguire Parkway,Oakland,CA,94609,1,1,37.8343,-122.2643
3,4,Holmes,Shimmings,99 Kennedy Court,Oakland,CA,94609,1,1,37.8343,-122.2643
4,5,Beverley,Gubbin,51 Mcbride Drive,Oakland,CA,94609,1,1,37.8343,-122.2643
...,...,...,...,...,...,...,...,...,...,...,...
8133,8134,Wilmette,Raittie,331 Sommers Park,San Geronimo,CA,94963,1,25,38.0138,-122.6703
8134,8135,Ammamaria,Newens,5 Esker Park,San Geronimo,CA,94963,1,25,38.0138,-122.6703
8135,8136,Blakeley,Verry,1947 Thackeray Road,San Geronimo,CA,94963,1,25,38.0138,-122.6703
8136,8137,Kikelia,Mendus,90777 Heath Crossing,San Geronimo,CA,94963,1,25,38.0138,-122.6703


In [55]:
# Creating a new column 'coordinates' with latitude and longitude in tuple format
customer_zip['coordinates'] = customer_zip.apply(lambda row: (row['latitude'], row['longitude']), axis=1)

# Display the updated DataFrame with the new 'coordinates' column
customer_zip


,customer_id,first_name,last_name,street,city,state,zip,closest_store_id,distance,latitude,longitude,coordinates
0,1,Robb,Weaving,5 Ramsey Place,Oakland,CA,94609,1,1,37.8343,-122.2643,"(37.8343, -122.2643)"
1,2,Robby,Belliard,6 Londonderry Plaza,Oakland,CA,94609,1,1,37.8343,-122.2643,"(37.8343, -122.2643)"
2,3,Sadella,Caudrelier,548 Mcguire Parkway,Oakland,CA,94609,1,1,37.8343,-122.2643,"(37.8343, -122.2643)"
3,4,Holmes,Shimmings,99 Kennedy Court,Oakland,CA,94609,1,1,37.8343,-122.2643,"(37.8343, -122.2643)"
4,5,Beverley,Gubbin,51 Mcbride Drive,Oakland,CA,94609,1,1,37.8343,-122.2643,"(37.8343, -122.2643)"
...,...,...,...,...,...,...,...,...,...,...,...,...
8133,8134,Wilmette,Raittie,331 Sommers Park,San Geronimo,CA,94963,1,25,38.0138,-122.6703,"(38.0138, -122.6703)"
8134,8135,Ammamaria,Newens,5 Esker Park,San Geronimo,CA,94963,1,25,38.0138,-122.6703,"(38.0138, -122.6703)"
8135,8136,Blakeley,Verry,1947 Thackeray Road,San Geronimo,CA,94963,1,25,38.0138,-122.6703,"(38.0138, -122.6703)"
8136,8137,Kikelia,Mendus,90777 Heath Crossing,San Geronimo,CA,94963,1,25,38.0138,-122.6703,"(38.0138, -122.6703)"


In [58]:
#Stations table

rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from stations

"""

station_zip = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
station_zip

,station,latitude,longitude,transfer_time
0,12th Street,37.803608,-122.272006,282
1,16th Street Mission,37.764847,-122.420042,287
2,19th Street,37.807869,-122.268980,67
3,24th Street Mission,37.752000,-122.418700,277
4,Antioch,37.996281,-121.783404,0
5,Ashby,37.853068,-122.269957,299
6,Balboa Park,37.721667,-122.447500,48
7,Bay Fair,37.697000,-122.126500,63
8,Berryessa,37.368361,-121.874655,288
9,Castro Valley,37.690748,-122.075679,0


In [59]:
# Creating a new column 'coordinates' with latitude and longitude in tuple format
station_zip['coordinates'] = station_zip.apply(lambda row: (row['latitude'], row['longitude']), axis=1)

# Display the updated DataFrame with the new 'coordinates' column
station_zip

,station,latitude,longitude,transfer_time,coordinates
0,12th Street,37.803608,-122.272006,282,"(37.803608, -122.272006)"
1,16th Street Mission,37.764847,-122.420042,287,"(37.764847, -122.420042)"
2,19th Street,37.807869,-122.268980,67,"(37.807869, -122.26898)"
3,24th Street Mission,37.752000,-122.418700,277,"(37.752, -122.4187)"
4,Antioch,37.996281,-121.783404,0,"(37.996281, -121.783404)"
5,Ashby,37.853068,-122.269957,299,"(37.853068, -122.269957)"
6,Balboa Park,37.721667,-122.447500,48,"(37.721667, -122.4475)"
7,Bay Fair,37.697000,-122.126500,63,"(37.697, -122.1265)"
8,Berryessa,37.368361,-121.874655,288,"(37.368361, -121.874655)"
9,Castro Valley,37.690748,-122.075679,0,"(37.690748, -122.075679)"


In [60]:
def my_calculate_distance(point_1, point_2):
    "Given two points in (latitude, longitude) format, calculate the distance between them in miles"
    
    geod = Geodesic.WGS84


    g = geod.Inverse(point_1[0], point_1[1], point_2[0], point_2[1])
    miles = g['s12'] / 1000 * 0.621371
    
    return miles

In [61]:
closest_stations = []
shortest_distances = []

# Loop through each customer's coordinates
for customer_coord in customer_zip['coordinates']:
    # Initialize variables for the closest station and shortest distance to this customer
    closest_station = None
    shortest_distance = float('inf')  # Initialize with a very large value

    # Loop through each train station's coordinates
    for station_coord in station_zip['coordinates']:
        # Calculate the distance between the customer and the station
        distance = my_calculate_distance(customer_coord, station_coord)
        
        # Update the closest station and shortest distance if the current station is closer
        if distance < shortest_distance:
            shortest_distance = distance
            closest_station = station_zip.loc[station_zip['coordinates'] == station_coord, 'station'].iloc[0]
    
    # Append the closest station and shortest distance to the respective lists
    closest_stations.append(closest_station)
    shortest_distances.append(shortest_distance)

# Add the results to the customer DataFrame
customer_zip['closest_station'] = closest_stations
customer_zip['shortest_distance'] = shortest_distances

# Display the updated customer DataFrame with the closest station and shortest distance
customer_zip.head(30)


,customer_id,first_name,last_name,street,city,state,zip,closest_store_id,distance,latitude,longitude,coordinates,closest_station,shortest_distance
0,1,Robb,Weaving,5 Ramsey Place,Oakland,CA,94609,1,1,37.8343,-122.2643,"(37.8343, -122.2643)",MacArthur,0.447226
1,2,Robby,Belliard,6 Londonderry Plaza,Oakland,CA,94609,1,1,37.8343,-122.2643,"(37.8343, -122.2643)",MacArthur,0.447226
2,3,Sadella,Caudrelier,548 Mcguire Parkway,Oakland,CA,94609,1,1,37.8343,-122.2643,"(37.8343, -122.2643)",MacArthur,0.447226
3,4,Holmes,Shimmings,99 Kennedy Court,Oakland,CA,94609,1,1,37.8343,-122.2643,"(37.8343, -122.2643)",MacArthur,0.447226
4,5,Beverley,Gubbin,51 Mcbride Drive,Oakland,CA,94609,1,1,37.8343,-122.2643,"(37.8343, -122.2643)",MacArthur,0.447226
5,6,Pavia,Millery,463 Columbus Pass,Oakland,CA,94609,1,1,37.8343,-122.2643,"(37.8343, -122.2643)",MacArthur,0.447226
6,7,Engracia,Jeanon,24 Hanover Court,Oakland,CA,94609,1,1,37.8343,-122.2643,"(37.8343, -122.2643)",MacArthur,0.447226
7,8,Melinda,Stodd,568 Nova Way,Oakland,CA,94609,1,1,37.8343,-122.2643,"(37.8343, -122.2643)",MacArthur,0.447226
8,9,Dinnie,Leek,38 Golf Alley,Oakland,CA,94609,1,1,37.8343,-122.2643,"(37.8343, -122.2643)",MacArthur,0.447226
9,10,Caitrin,Filipowicz,61 Michigan Drive,Oakland,CA,94609,1,1,37.8343,-122.2643,"(37.8343, -122.2643)",MacArthur,0.447226


In [63]:
closest_station_counts = customer_zip['closest_station'].value_counts().reset_index()
closest_station_counts.columns = ['closest_station', 'count']
closest_station_df = pd.DataFrame(closest_station_counts)
total_count = closest_station_counts['count'].sum()
closest_station_counts['percentage_of_count'] = closest_station_counts['count'] / total_count * 100

closest_station_df

,closest_station,count,percentage_of_count
0,El Cerrito del Norte,831,10.211354
1,Richmond,538,6.610961
2,Rockridge,494,6.070288
3,Civic Center,468,5.750799
4,Downtown Berkeley,463,5.689359
5,Fruitvale,440,5.406734
6,El Cerrito Plaza,409,5.025805
7,MacArthur,385,4.730892
8,Lake Merritt,375,4.608012
9,19th Street,337,4.141067


In [64]:
# Group closest station by community
merged_closest_station = closest_station_df.merge(final_df[['name', 'community']], left_on='closest_station', right_on='name', how='inner')
merged_closest_station


,closest_station,count,percentage_of_count,name,community
0,El Cerrito del Norte,831,10.211354,El Cerrito del Norte,15
1,Richmond,538,6.610961,Richmond,15
2,Rockridge,494,6.070288,Rockridge,31
3,Civic Center,468,5.750799,Civic Center,29
4,Downtown Berkeley,463,5.689359,Downtown Berkeley,15
5,Fruitvale,440,5.406734,Fruitvale,25
6,El Cerrito Plaza,409,5.025805,El Cerrito Plaza,15
7,MacArthur,385,4.730892,MacArthur,15
8,Lake Merritt,375,4.608012,Lake Merritt,25
9,19th Street,337,4.141067,19th Street,15


In [65]:
# Group closest station by community
merged_closest_station = closest_station_df.merge(final_df[['name', 'community']], left_on='closest_station', right_on='name', how='inner')
grouped_by_community = merged_closest_station.groupby('community')
community_counts = grouped_by_community.size().reset_index(name='customer_count')
print(community_counts)


   community  customer_count
0         14               5
1         15               7
2         23               4
3         25               9
4         29               5
5         31               8
6         42               5


In [66]:
final_groupby = merged_closest_station.groupby('community').apply(lambda x: x.sort_values(by='percentage_of_count', ascending=False))
display_final_groupby = final_groupby.reset_index(drop=True)
final_groupby_df = pd.DataFrame(display_final_groupby)
final_groupby_df

,closest_station,count,percentage_of_count,name,community
0,16th Street Mission,198,2.433030,16th Street Mission,14
1,Balboa Park,157,1.929221,Balboa Park,14
2,24th Street Mission,145,1.781765,24th Street Mission,14
3,Glen Park,127,1.560580,Glen Park,14
4,Daly City,37,0.454657,Daly City,14
5,El Cerrito del Norte,831,10.211354,El Cerrito del Norte,15
6,Richmond,538,6.610961,Richmond,15
7,Downtown Berkeley,463,5.689359,Downtown Berkeley,15
8,El Cerrito Plaza,409,5.025805,El Cerrito Plaza,15
9,MacArthur,385,4.730892,MacArthur,15


Sorting Centrality Algorithms results

In [70]:
# Analysis on harmonic centrality
# Stations with harmonic degree centrality and exits
harmonic_final_df = final_df[['name','harmonic', 'population_within_1mile', 'entries','exits','community']]
harmonic_sort1 = harmonic_final_df.sort_values(by=['harmonic','exits'],ascending=[False, False])
harmonic_sort1.head(30)

,name,harmonic,population_within_1mile,entries,exits,community
10,12th Street,0.233302,16062,119623,115837,15
12,Lake Merritt,0.231867,16062,71296,74767,25
0,West Oakland,0.228991,26254,93142,84396,29
13,MacArthur,0.226052,22811,94830,91758,15
14,19th Street,0.219525,16062,113251,107926,15
5,Coliseum,0.216537,0,80998,84167,25
16,Fruitvale,0.214852,52299,95590,94081,25
11,Bay Fair,0.209259,41059,62762,63233,25
6,Embarcadero,0.204956,54398,370182,420668,29
17,San Leandro,0.204085,48088,76414,77476,25


In [71]:
# Stations with harmonic harmonic centrality and population
harmonic_final_df = final_df[['name','harmonic', 'population_within_1mile', 'entries','exits','community']]
harmonic_sort2 = harmonic_final_df.sort_values(by=['harmonic','population_within_1mile'],ascending=[False, False])
harmonic_sort2.head(30)

,name,harmonic,population_within_1mile,entries,exits,community
10,12th Street,0.233302,16062,119623,115837,15
12,Lake Merritt,0.231867,16062,71296,74767,25
0,West Oakland,0.228991,26254,93142,84396,29
13,MacArthur,0.226052,22811,94830,91758,15
14,19th Street,0.219525,16062,113251,107926,15
5,Coliseum,0.216537,0,80998,84167,25
16,Fruitvale,0.214852,52299,95590,94081,25
11,Bay Fair,0.209259,41059,62762,63233,25
6,Embarcadero,0.204956,54398,370182,420668,29
17,San Leandro,0.204085,48088,76414,77476,25


In [72]:
# Analysis on betweenness centrality
# Stations with betweenness betweenness centrality and exits
betweenness_final_df = final_df[['name','betweenness', 'population_within_1mile', 'entries','exits','community']]
betweenness_sort1 = betweenness_final_df.sort_values(by=['betweenness','exits'],ascending=[False, False])
betweenness_sort1.head(30)

,name,betweenness,population_within_1mile,entries,exits,community
13,MacArthur,1176.0,22811,94830,91758,15
10,12th Street,1116.0,16062,119623,115837,15
14,19th Street,1088.0,16062,113251,107926,15
12,Lake Merritt,1020.0,16062,71296,74767,25
16,Fruitvale,980.0,52299,95590,94081,25
0,West Oakland,980.0,26254,93142,84396,29
5,Coliseum,960.0,0,80998,84167,25
6,Embarcadero,936.0,54398,370182,420668,29
8,Montgomery Street,888.0,85465,295434,336738,29
9,Powell Street,836.0,140730,337468,304277,29


In [73]:
# Stations with betweenness betweenness centrality and population
betweenness_final_df = final_df[['name','betweenness', 'population_within_1mile', 'entries','exits','community']]
betweenness_sort2 = betweenness_final_df.sort_values(by=['betweenness','population_within_1mile'],ascending=[False, False])
betweenness_sort2.head(30)

,name,betweenness,population_within_1mile,entries,exits,community
13,MacArthur,1176.0,22811,94830,91758,15
10,12th Street,1116.0,16062,119623,115837,15
14,19th Street,1088.0,16062,113251,107926,15
12,Lake Merritt,1020.0,16062,71296,74767,25
16,Fruitvale,980.0,52299,95590,94081,25
0,West Oakland,980.0,26254,93142,84396,29
5,Coliseum,960.0,0,80998,84167,25
6,Embarcadero,936.0,54398,370182,420668,29
8,Montgomery Street,888.0,85465,295434,336738,29
9,Powell Street,836.0,140730,337468,304277,29
